In [ ]:
## importamos algumas bibliotecas que serão usadas
import numpy as np  # Para um mínimo de matemática

# MAP5729 - EP3: MMQ


**Nome:** Narayan Shimanoe Lisboa

**Número USP:** 14600141

**Turma:** 02

<p align = "justify">O objetivo deste EP é implementar o método dos mínimos quadrados. Focaremos na resolução de um sistema sobredeterminado, resolvendo um problema relacionado ao sistema de GPS. Esse problema, na realidade, é descrito por um sistema não-linear, mas consideraremos uma forma linearizada, permitindo a aplicação do MMQ. Aproveitaremos para introduzir e implementar a fatoração QR, que permite resolver o sistema normal de forma mais estável comparado à eliminação de Gauss ou fatoração LU.</p>

**Orientações:**
- Salve uma cópia deste notebook para editá-lo.
- Utilize o índice à esquerda para identificar as questões que precisam ser respondidas.
- As respostas das questões deverão ser respondidas nos espaços indicados abaixo.
- Envie o link do seu notebook na página de entrega da atividade no Moodle da disciplina.
- Data limite de entrega: 05/05

<font color='red'>
IMPORTANTE: o problema descrito neste EP depende de dados dependentes de um parâmetro $i_{rec}$, escolhido em função de seu número USP. $i_{rec}$ é calculado como o resto da divisão do seu número USP por $4$.

Complete e rode a célula abaixo indicando seu valor de $i_{rec}$.
</font>



In [ ]:
nusp = 14600141
irec = nusp%4
print(irec)

1


#Parte 1 - Formulação do problema


## O sistema de posicionamento global

<p align = "justify">O Sistema de Posicionamento Global (GPS, sigla do nome em inglês) é um
sistema de navegação formado por uma rede de pelo menos 24 satélites orbitando a Terra e
transmitindo sinais para os receptores GPS, que usam triangulação para calcular
a localização do usuário. Na descrição a seguir, o sistema de coordenadas tem
sua origem $O$ no centro da Terra, o eixo $Oz$ positivo aponta na direção do Polo
Norte, o plano $Oxy$ é o plano do equador com o eixo $Ox$ positivo cortando o
meridiano de Greenwich e o eixo $Oy$ positivo corta o meridiano de longitude
$90^{\circ}$E.</p>

<p align = "justify">Um dado satélite $i$ transmite a sua posição atual $(x_i , y_i , z_i)$ e o instante $te_i$
de emissão do sinal. O receptor GPS grava esta informação juntamente com
o instante $tr_i$ da recepção do sinal. Entretanto, o relógio do receptor é menos
preciso do que o relógio do satélite, havendo um erro de sincronia $T$ . O instante
correto da recepção é $tr_i - T$ . Se $T > 0$ o relógio do receptor está adiantado e
se $T < 0$ ele está atrasado.</p>

<p align = "justify">Portanto, o tempo que o sinal leva do satélite ao receptor é $t_i - T$ , onde
$t_i = tr_i - te_i$ é o lapso de tempo aparente, e o receptor deve estar na superfı́cie
da esfera de raio $c(t_i - T)$ e centro $(x_i , y_i , z_i)$, onde $c = 299792458$ m/s é a
velocidade da luz no vácuo. As coordenadas $(x, y, z)$ do receptor devem então
satisfazer a relação</p>

\begin{equation*}
  (x - x_i)^2 + (y - y_i)^2 + (z - z_i)^2 = (w_i - w)^2
\end{equation*}

<p align = "justify">em que $w_i = ct_i$ e $w = cT$ (ao multiplicarmos por c, ficamos apenas com dimensões
espaciais).</p>

<p align = "justify">Em qualquer instante, dados de 5 a 8 satélites são obtidos pelo receptor em
qualquer ponto da terra. Então, assumindo que temos dados de $n > 4$ satélites,
as 4 incógnitas $(x, y, z, w)$ devem satisfazer as equações</p>

\begin{equation}
  r_i(x,y,z,w) = (x - x_i)^2 + (y - y_i)^2 + (z - z_i)^2 - (w_i - w)^2 = 0, \tag{1}
\end{equation}

<p align = "justify">$1 \le i \le n$, que formam um sistema não-linear sobredeterminado. Como as
medidas estão sujeitas a erros, este sistema em geral não tem solução (se não
houvesse erros, bastaria resolver 4 das n equações).</p>


<p align = "justify">Observe que (1) é um sistema de equações não-lineares, já que ele possui termos quadráticos nas incógnitas $x$, $y$, $z$ e $w$. Sua resolução exigiria a utilização de métodos iterativos, como o método de Newton. No entanto, se subtrairmos a equação $n$ de cada uma das outras equações,</p>

\begin{equation}
  r_i(x,y,z,w) - r_n(x,y,z,w) = 0,\quad 1 \le i \le n-1 \tag{2}
\end{equation}

<p align = "justify">os termos quadráticos cancelam e obtemos um sistema linear sobredeterminado $A \mathbf{x} = \mathbf{b}$, com $A \in \mathbb{R}^{(n-1)\times4}$. A solução desse sistema linear poderia ser usada como chute inicial para a resolução iterativa do sistema não-linear (1). No entanto, vamos resolver neste EP apenas o problema linear.</p>

## Questão 1.1.

Escreva, para a linha $i$ do sistema linear sobredeterminado (2), os quatro elementos $a_{ij}$ de $A$ e o elemento $b_i$ de $\mathbf{b}$:

**Resposta**:
- $a_{i1} = 2(x_n-x_i)$
- $a_{i2} = 2(y_n-y_i)$
- $a_{i3} = 2(z_n-z_i)$
- $a_{i4} = -2(w_n-w_i)$
- $b_i = x_n^2- x_i^2  + y_n^2 - y_i^2 + z_n^2- z_i^2  + w_i^2 - w_n^2$

# Parte 2 - Fatoração QR

**Motivação**

Aplicando o método dos mínimos quadrados ao sistema linear sobredeterminado (2), obtemos o sistema linear normal $A^T A \mathbf{x} = A^T \mathbf{b}$, de tamanho igual ao número de incógnitas (neste caso, 4). No entanto, geralmente não é recomendado resolver esse sistema diretamente (usando, por exemplo, Eliminação de Gauss), pois a matriz $A^TA$ pode ser muito **mal condicionada**. Isso significa que o sistema normal pode ser sensível a erros de arredondamento, conduzindo a soluções numéricas pouco precisas. Mesmo que isso talvez não se verifique no exemplo que estamos resolvendo, vamos apresentar e implementar uma outra técnica menos suscetível a erros de arredondamento.

**A fatoração QR**

Seja $A \in \mathbb{R}^{N \times M}$. Se $N \geq M$, a matriz $A$ pode ser escrita na forma $A = QR$, onde $Q \in \mathbb{R}^{N \times M}$ satisfaz $Q^T Q = I$ (ou seja, as colunas de $Q$ são ortonormais) e $R \in \mathbb{R}^{M \times M}$ é uma matriz triangular superior. Essa é a chamada **fatoração QR** de $A$.

Substituindo $A = QR$ em $A^T A \mathbf{x} = A^T \mathbf{b}$, temos

\begin{equation}
  \tag{3}
  R^T Q^T Q R \mathbf{x} = R^T Q^T \mathbf{b} \iff R^T R \mathbf{x} = R^T Q^T \mathbf{b} \iff (R^T)^{-1}R^T R \mathbf{x} = (R^T)^{-1}R^T Q^T \mathbf{b} \iff R \mathbf{x} = Q^T \mathbf{b}
\end{equation}

A matriz $R$ possui inversa se as colunas de $A$ são linearmente independentes.

A equação (3) mostra que, usando a fatoração $QR$, a solução do sistema normal é dada pela solução de um **sistema triangular superior** (3), que pode ser facilmente resolvido por subsituição.


**Cálculo da fatoração QR**

As matrizes $Q$ e $R$ podem ser determinadas aplicando o processo de **ortonormalização de Gram-Schmidt** às colunas $\mathbf{a}_j$, $j = 1, \dots, M$, de $A$. Esse método consiste em construir sucessivamente vetores $\mathbf{q}_j$, $j = 1, \dots, M$ (as colunas de $Q$), de modo que cada um desses vetores seja ortogonal aos vetores previamente construídos. Além disso, os vetores são normalizados, de forma que $||\mathbf{q}_j||_2 = \sqrt{\langle \mathbf{q}_j, \mathbf{q}_j \rangle} = 1$, $j = 1, \dots, M$.


O algoritmo é descrito da seguinte forma: começamos determinando a primeira coluna de $Q$, $\mathbf{q}_1 = \frac{1}{||\mathbf{a}_1||_2} \mathbf{a}_1$, e em seguida, determinamos cada coluna $j = 2, \dots, M$, de $Q$, dada pela diferença entre $\mathbf{a}_j$ e a projeção de $\mathbf{a}_j$ no espaço gerado pelos vetores $\mathbf{q}_1, \dots, \mathbf{q}_j$ já construídos. Pode-se mostrar que os coeficientes dessa projeção são os elementos de $R$. Assim, temos o algoritmo:

**para** $j = 1, \dots, M$, **faça**:
    
  * $\mathbf{q}_j = \mathbf{a}_j$
  * **para** $ i =1, \dots, j-1$:

   -   $r_{ij} = \langle \mathbf{q}_i, \mathbf{a}_j \rangle$        
   -   $\mathbf{q}_j \leftarrow \mathbf{q}_j - r_{ij} \mathbf{q}_i$
  
  * $r_{jj} = ||\mathbf{q}_j||_2$
  * se $r_{jj} = 0$: PARE (as colunas de $A$ são linearmente dependentes)
  * $\mathbf{q}_j \leftarrow \mathbf{q}_j / r_{jj}$

**fim**
    

**Observação:** o próprio algoritmo de Gram-Schmidt também pode estar sujeito a instabilidades numéricas devido a erros de arredondamento, caso as colunas de $A$ sejam quase linearmente dependentes, em especial para matrizes de grande dimensão. Existem outros métodos para calcular a fatoração QR (por exemplo, usando matrizes de Houselhoder e rotações de Givens), mas eles não serão vistos aqui.

## Questão 2.1.

Vamos começar escrevendo algumas funções necessárias para calcular a fatoração QR de $A$ e para a resolução do sistema normal usando a fatoração QR. Complete as funções abaixo:

<!-- - Função *produto_interno*: calcula o produto interno usual em $\mathbb{R}^M$: $\langle u,v \rangle = \sum_{i=1}^M u_iv_i$
- Função *produto_matriz_vetor*: calcula o produto entre $A \in \mathbb{R}^{N \times M}$ e $\mathbf{x} \in \mathbb{R}^M$. Lembrando que, em Python, $A[i,:]$ extrai a linha $i$ de $A$, você pode usar chamar a função *produto_interno* dentro desta função.
- Função *produto_matriz_matriz*: calcula o produto entre $A \in \mathbb{R}^{N \times M}$ e $\mathbf{x} \in \mathbb{R}^M$. Lembrando que, em Python, $A[i,:]$ extrai a linha $i$ de $A$, você pode usar chamar a função *produto_interno* dentro desta função. -->
- Função *norma*: calcula a norma Euclidiana (norma 2) de um vetor.
- Função *gram_schmidt*: implementa o algoritmo de Gram-Schmidt descrito acima.
- Função *resolver_sistema_triangular_superior*: resolve um sistema linear triangular superior.

Para implementar o produto interno entre vetores e a multiplicação entre vetores, você pode usar a função *np.dot* da biblioteca *numpy*:

- np.dot(u,v): produto interno entre os vetores $u, v \in \mathbb{R}^{n}$
- np.dot(A,u): produto entre a matriz $A \in \mathbb{R}^{n \times m}$ e o vetor $u \in \mathbb{R}^m$

**Atenção:** u*v calcula o produto componente a componente (não é o produto interno!)

Além disso, a transposta de uma matriz $A$ á dada por:
- A.T, ou np.transpose(A)


In [ ]:
def norma(v):
  """
  Calcula a norma Euclidiana (norma 2) de um vetor

  Entradas:
  - v: vetor de tamanho M

  Saídas:
  - norma: norma de v
  """

  norma = np.sqrt(np.dot(v,v))

  return norma

def gram_schmidt(A):
  """
  Aplica o processo de ortonormalização de Gram-Schmidt às colunas de A

  Entradas:
  - A: matriz de tamanho N x M

  Saídas:
  - Q: matriz de tamanho N x M, satisfazendo Q^T Q = I
  - R: matriz triangular de tamanho M x M
  """

  N,M = A.shape             ## número de linhas e de colunas de A
  Q = np.zeros((N,M))       ## inicializa a matriz Q com zeros
  R = np.zeros((M,M))       ## inicializa a matriz R com zeros

  ## Coluna j de Q
  for j in range(0,M):
    q = A[:,j]
    ## COMPLETAR
    for i in range(0,j):
      R[i,j] = np.dot(Q[:,i],A[:,j])
      q = q - R[i,j]*Q[:,i]
    R[j,j] = norma(q)
    if R[j,j] == 0:
      print("As colunas de A são linearmente dependentes")
      return
    Q[:,j] = q/R[j,j]

  return Q,R

def resolver_sistema_triangular_superior(R,z):
  """
  Resolve o sistema triangular superior Rx = z

  Entradas:
  - R: matriz triangular de tamanho M x M
  - z: vetor de tamanho M

  Saídas:
  - x: vetor de tamanho M
  """

  M = R.shape[0]        ## número de linhas (e de colunas) de R
  x = np.zeros(M)       ## inicializa a solução com zeros
  (n, k) = R.shape


  z_copy = z.copy()

  for i in range(n - 1, -1, -1):
    for j in range(k - 1, i, -1):
      z_copy[i] = z_copy[i] - x[j] * R[i, j]  # Use the copy
    x[i] = z_copy[i] / R[i, i]  # Use the copy

  return x

## Questão 2.2

Vamos inicialmente validar a função implementando o algoritmo de Gram-Schmidt. Considere

\begin{equation}
  A = \left(\begin{array}{c c c}
  1 & 0\\
  1 & 1 \\
  0 & -1
  \end{array}\right)
\end{equation}

Verifique que sua função fornece

\begin{equation}
  Q = \left(\begin{array}{c c c}
  \frac{1}{\sqrt{2}} & -\frac{1}{\sqrt{6}}\\
  \frac{1}{\sqrt{2}} & \frac{1}{\sqrt{6}}\\
  0 & -\frac{2}{\sqrt{6}}
  \end{array}\right), \qquad
  R = \left(\begin{array}{c c}
  \sqrt{2} & \frac{1}{\sqrt{2}} \\
  0 & \frac{3}{\sqrt{6}}
  \end{array} \right)
\end{equation}

Além disso, utilize a fatoração QR para resolver o sistema sobredeterminado

\begin{equation}
  \left(\begin{array}{c c c}
  1 & 0\\
  1 & 1 \\
  0 & -1
  \end{array}\right)
  \left(
    \begin{array}{c}
    x_1\\x_2
    \end{array}
  \right)
  =
  \left(
    \begin{array}{c}
    1\\1\\1
    \end{array}
  \right)
\end{equation}

e verifique que sua implementação fornece o mesmo resultado (qual?) obtido resolvendo diretamente o sistema normal, como visto em aula.

In [ ]:
## Matriz A
A = np.array([[1,0],[1,1],[0,-1]],dtype=float)

## Resultados de referência
Q_ref = np.array([[1/np.sqrt(2),-1/np.sqrt(6)],[1/np.sqrt(2),1/np.sqrt(6)],[0,-2/np.sqrt(6)]])
R_ref = np.array([[np.sqrt(2),1/np.sqrt(2)],[0,3/np.sqrt(6)]])
x_ref = np.matmul(Q_ref.T,np.array([1,1,1]))


## Calcular fatoração QR de A
# COMPLETAR
Q,R = gram_schmidt(A)

## Resolver Ax = b por MMQ
# COMPLETAR
x = resolver_sistema_triangular_superior(R,np.matmul(Q.T,np.array([1,1,1])))

## Imprimir e comparar resultados
print("Q: \n{}".format(Q))
print("R: \n{}".format(R))
print("x: \n{}".format(x))
print("")
print("Q_ref: \n{}".format(Q_ref))
print("R_ref: \n{}".format(R_ref))
print("x_ref: \n{}".format(x_ref))
print("")
print("Q - Q_ref: \n{}".format(Q-Q_ref))
print("R - R_ref: \n{}".format(R-R_ref))
print("x - x_ref: \n{}".format(x-x_ref))

Q: 
[[ 0.70710678 -0.40824829]
 [ 0.70710678  0.40824829]
 [ 0.         -0.81649658]]
R: 
[[1.41421356 0.70710678]
 [0.         1.22474487]]
x: 
[ 1.33333333 -0.66666667]

Q_ref: 
[[ 0.70710678 -0.40824829]
 [ 0.70710678  0.40824829]
 [ 0.         -0.81649658]]
R_ref: 
[[1.41421356 0.70710678]
 [0.         1.22474487]]
x_ref: 
[ 1.41421356 -0.81649658]

Q - Q_ref: 
[[0.00000000e+00 1.11022302e-16]
 [0.00000000e+00 5.55111512e-17]
 [0.00000000e+00 0.00000000e+00]]
R - R_ref: 
[[ 0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -2.22044605e-16]]
x - x_ref: 
[-0.08088023  0.14982991]


# Parte 3 - Resolução do problema

Já temos implementadas todas as ferramentas necessárias para a resolução do problema linearizado do GPS usando MMQ.

Vamos inicialmente obter os dados do problema. Estão disponíveis dados de quatro receptores, e você utilizará aqules correspondentes ao receptor $i_{rec}$, com $i_{rec}$ definido em função de seu número USP. Para cada receptor, consideraremos a posição fornecida por $n = 7$ satélites. Rode o código abaixo para fazer o download do arquivo com os dados de entrada e para obter os dados correspondentes ao seu valor de $i_{rec}$. Cada linha do arquivo corresponde a um satélite $i$ e possui quatro colunas, correspondendo às suas coordenadas $x_i,y_i,z_i,w_i$.  

In [ ]:
# Leitura do arquivo input_gps_{i_rec}.txt, salvando o conteúdo em input_gps
import gdown

# nomes dos arquivos
file_ids = [
              '1kpZoDYPCW-f947rGwtaPZ74RXn5RdbVg',
              '1aQwQUD5LATaeqz7CUb2IysWl-jZJeIDM',
              '1giXyeQx0WYh3rj43yckhIy_WKaNAuORn',
              '1JIRyDnmNdz1TA4wmYnHIaoFsyltZZI62'
           ]

# seu arquivo
file_id = file_ids[irec]

## url
url = f'https://drive.google.com/uc?id={file_id}&confirm=t'

# Download the file
gdown.download(url, 'input_gps.txt', quiet=False)

# Load it with NumPy
import numpy as np
meus_dados = np.loadtxt('input_gps.txt')
print()

# Impressão (restringindo a precisão para caber na tela)
for row in meus_dados:
    # As duas primeiras colunas são inteiros
    print(f"{int(row[0]):3d} {int(row[1]):3d} ", end="")

    # Demais colunas com sete algarismos significativos
    print(" ".join(f"{val:15.6e}" for val in row[2:]))

Downloading...
From: https://drive.google.com/uc?id=1aQwQUD5LATaeqz7CUb2IysWl-jZJeIDM&confirm=t
To: /content/input_gps.txt
100%|██████████| 693/693 [00:00<00:00, 961kB/s]


-15237643 -6561298    2.091441e+07    3.069011e+07
-3444440 -26380107    2.894201e+06    2.423388e+07
2912486 -25014385    8.546328e+06    2.382581e+07
-13075174 -16895548    1.569555e+07    2.826600e+07
10367775 -12758757    2.119768e+07    2.627340e+07
21159876 -16088326    1.981508e+05    2.128391e+07
21438233 -1837413    1.586470e+07    2.573184e+07


## Questão 3.1.

Utilizando os dados obtidos, construa a matriz $A$ e o vetor $\mathbf{b}$ do sistema linear sobredeterminado $A \mathbf{x} = \mathbf{b}$.


In [ ]:
## Inicializa a matriz e o vetor com zeros
A = np.zeros((meus_dados.shape[0]-1,4))
b = np.zeros(meus_dados.shape[0]-1)

## Preencher a linha i do sistema linear
for i in range(meus_dados.shape[0]-1):
  A[i,0] = 2*(meus_dados[meus_dados.shape[0]-1,0] - meus_dados[i,0])## COMPLETAR
  A[i,1] = 2*(meus_dados[meus_dados.shape[0]-1,1] - meus_dados[i,1])## COMPLETAR
  A[i,2] = 2*(meus_dados[meus_dados.shape[0]-1,2] - meus_dados[i,2])## COMPLETAR
  A[i,3] = -2*(meus_dados[meus_dados.shape[0]-1,3] - meus_dados[i,3])## COMPLETAR
  b[i]   = meus_dados[meus_dados.shape[0]-1,0]**2 - meus_dados[i,0]**2 + meus_dados[meus_dados.shape[0]-1,1]**2 - meus_dados[i,1]**2 + meus_dados[meus_dados.shape[0]-1,2]**2 - meus_dados[i,2]**2 + meus_dados[i,3]**2 - meus_dados[meus_dados.shape[0]-1,3]**2 ## COMPLETAR

## Imprime a matriz e o vetor
print("A: \n{}".format(A))
print("b: \n{}".format(b))

A: 
[[ 73351752.978        9447770.984      -10099411.678
    9916552.27687944]
 [ 49765346.742       49085389.36        25940998.822
   -2995900.87672862]
 [ 37051492.904       46353943.906       14636746.372
   -3812054.46337759]
 [ 69026815.324       30116271.8           338300.642
    5068320.36021337]
 [ 22140915.106       21842689.83       -10665953.628
    1083136.18480674]
 [   556713.884       28501826.104       31333099.952
   -8895855.71062995]]
b: 
[ 2.81769423e+14 -7.63341788e+13 -8.70373190e+13  1.48731733e+14
  2.32088216e+13 -2.01073839e+14]


## Questão 3.2.

Calcule a fatoração QR de $A$ e utilize-a para resolver o sistema normal. A solução do sistema normal é um vetor $\mathbf{x} = (x,y,z,w)^T$, contendo as coordenadas do receptor.

In [ ]:
## Resolver o MMQ usando QR
Q,R = gram_schmidt(A)
x = resolver_sistema_triangular_superior(R,np.matmul(Q.T,b))

## Coordenadas do receptor
xr = x[0]
yr = x[1]
zr = x[2]
wr = x[3]

print("x: {}".format(xr))
print("y: {}".format(yr))
print("z: {}".format(zr))
print("w: {}".format(wr))

x: 4003121.614986875
y: -4251812.717302605
z: -2546900.9979677536
w: 260312.01282756473


## Questão 3.3.

As coordenadas $(x,y,z)$ do receptor devem ser tais que $\sqrt{x^2 + y^2 + z^2}$ esteja próximo de uma determinada constante. Que constante é essa? Verifique que a solução obtida acima satisfaz essa propriedade, para checar que seus resultados são coerentes.

**Resposta**:
- Significado físico da constante:
- Valor da constante:
- Valor obtido:

In [ ]:
## COMPLETAR
sqrtr = np.sqrt(xr**2 + yr**2 + zr**2)
print(sqrtr)

6370996.683474291


## Questão 3.4.

Usando a solução estimada $\mathbf{x}$, determine $\max_{1 \leq i \leq n-1} |\tilde{r}_i|$ e $\max_{1 \leq i \leq n-1} |r_i|$, onde $\tilde{\mathbf{r}} := \mathbf{b} - A \mathbf{x}$ é o resíduo do sistema linear (2) e $\mathbf{r}$ é o resíduo do sistema não-linear (este último dado pelas equações (1)). Explique em poucas palavras por que esses valores não são nulos.

**Resposta:**

In [ ]:
## Calcular vetor rtil
rtil = b - np.matmul(A,x)
print(np.max(np.abs(rtil)))

## Calcular vetor r
r = np.zeros(meus_dados.shape[0]-1)
for i in range(meus_dados.shape[0]-1):
  r[i] = (meus_dados[i,0]-xr)**2 + (meus_dados[i,1]-yr)**2 + (meus_dados[i,2]-zr)**2 - (meus_dados[i,3]-wr)**2
print(np.max(np.abs(r)))

17265431.40625
1021338145.0


## Questão 3.5.

Determine o erro de sincronia $T = w / c$ do receptor

In [ ]:
T = wr/299792458
print("T: \n{}".format(T))

T: 
0.0008683074102803637


## Questão 3.6.

Utilizando a solução estimada $\mathbf{x}$ e assumindo que a Terra seja uma esfere perfeita, determine a latitude e a longitude (coordenadas geográficas) do receptor (como isso pode ser feito?) e localize-o no globo terrestre.

**Resposta**:
- Latitude:
- Longitude:
- Localização:

In [ ]:
## COMPLETAR
lat = np.arcsin(zr/sqrtr) * 180/np.pi
lon = np.arctan2(yr,xr) * 180/np.pi

print("Latitude: {}".format(lat))
print("Longitude: {}".format(lon))

Latitude: -23.563483497768832
Longitude: -46.72559121217399
